In [1]:
import os
import rasterio
import numpy as np

In [2]:
import sys
sys.path.append('..')

In [3]:
import config

import rsutils.utils as utils
import resample_worldcereal_cropmasks

In [ ]:
config.FOLDERPATH_OUTPUTS

In [5]:
aez_s2_grid_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 's2_grid_level/worldcereal/')
s2_grid_level_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 's2_grid_level/aggregated_worldcereal/')
merged_product_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 's2_grid_level/aggregated_worldcereal/')
resampled_cropmasks_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 's2_grid_level/resampled_cropmasks/')
output_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 'merged_WC_cropmask/')

In [6]:
WC_S2_GRID_RES = 4

WC_PRODUCTS_MERGE_DICT = {
    # 'cereals': ['tc-springcereals_springcereals', 'tc-wintercereals_wintercereals'] # JOB_NAME: 'France'
    'maize': ['tc-maize-main_maize', 'tc-maize-second_maize'] # JOB_NAME: 'south-east-africa'
}

WC_OUTPRODUCTS = [
    'maize', 
    # 'tc-annual_temporarycrops',
]
BINARY_MASK_SETTINGS = {
    'maize': {
        'threshold': 10,
        'how': 'gte',
    },
    'tc-annual_temporarycrops': {
        'threshold': 0,
        'how': 'gt',
    }
}

In [ ]:
# 49 m 12.2 s
merged_cropmask_catalogue_df = \
resample_worldcereal_cropmasks.resample_worldcereal_cropmasks(
    roi_geom_filepath = config.FILEPATH_ROI_SHAPE,
    worldcereal_folderpath = config.FOLDERPATH_WORLDCEREAL,
    worldcereal_aez_filepath = config.FILEPATH_WORLDCEREAL_AEZ,
    ref_tif_filepath = config.FILEPATH_REFERENCE_GEOTIFF,
    # ref_tif_filepath = '/gpfs/data1/cmongp2/sasirajann/nh_crop_calendar/crop_calendar/data/outputs/ref_mod09.ndvi.global_0.05_degree.2019.001.c6.v1.tif',
    aez_s2_grid_folderpath = aez_s2_grid_folderpath,
    s2_grid_level_folderpath = s2_grid_level_folderpath,
    merged_product_folderpath = merged_product_folderpath,
    resampled_cropmasks_folderpath = resampled_cropmasks_folderpath,
    output_folderpath = output_folderpath,
    merge_products_dict = WC_PRODUCTS_MERGE_DICT,
    out_products = WC_OUTPRODUCTS,
    s2_grid_res = WC_S2_GRID_RES,
    overwrite = False,
)

In [ ]:
merged_cropmask_catalogue_df

In [ ]:
tif_filepaths = dict(zip(
    merged_cropmask_catalogue_df['product'],
    merged_cropmask_catalogue_df['tif_filepath'],
))

tif_filepaths

In [ ]:
os.makedirs(config.FOLDERPATH_MASKS, exist_ok=True)

In [ ]:
def create_binary_mask(
    tif_filepath:str,
    out_mask_filepath:str,
    min_threshold,
    how='gte',
):
    with rasterio.open(tif_filepath) as src:
        ndarray = src.read()
        meta = src.meta.copy()
    
    out_ndarray = np.zeros(shape=ndarray.shape)
    if how == 'gte':
        out_ndarray[ndarray >= min_threshold] = 1
    elif how == 'gt':
        out_ndarray[ndarray > min_threshold] = 1
    else:
        raise ValueError(f"how must be either 'gte' or 'gt'.")

    _folderpath = os.path.split(out_mask_filepath)[0]
    os.makedirs(_folderpath, exist_ok=True)

    meta['dtype'] = rasterio.uint8
    meta['nodata'] = 0

    with rasterio.open(out_mask_filepath, 'w', **meta) as dst:
        dst.write(out_ndarray)

In [ ]:
binary_mask_filepaths = {}

for key, binmask_config in BINARY_MASK_SETTINGS.items():
    tif_filepath = tif_filepaths[key]
    out_mask_filepath = utils.modify_filepath(
        filepath=tif_filepath,
        new_folderpath=config.FOLDERPATH_MASKS,
    )
    binary_mask_filepaths[key] = out_mask_filepath
    create_binary_mask(
        tif_filepath=tif_filepath, 
        out_mask_filepath=  out_mask_filepath,
        min_threshold=binmask_config['threshold'],
        how=binmask_config['how'],
    )

In [ ]:
geoglam_maize_geotiff = '../data/GEOGLAM-BACS_v1.0.0/Percent_Maize.tif'

In [ ]:
import geopandas as gpd
import rasterio.warp

In [ ]:
def crop_and_save(
    src_filepath:str,
    dst_filepath:str,
    shapes_gdf:gpd.GeoDataFrame,
):
    out_ndarray, out_meta = utils.crop_tif(
        src_filepath = src_filepath,
        shapes_gdf = shapes_gdf,
    )

    with rasterio.open(dst_filepath, 'w', **out_meta) as dst:
        dst.write(out_ndarray)


def crop_and_resample(
    ref_filepath:str,
    src_filepath:str,
    dst_filepath:str,
    shapes_gdf:gpd.GeoDataFrame,
    resampling = rasterio.warp.Resampling.average,
    src_nodata = None,
    dst_nodata = None,
    dst_dtype = None,
    working_dir:str = None,
    delete_temp_files:bool = True,
):
    cropped_ref_filepath = utils.add_epochs_prefix(
        filepath = ref_filepath,
        prefix = 'cropped_',
        new_folderpath = working_dir,
    )

    crop_and_save(
        src_filepath = ref_filepath,
        dst_filepath = cropped_ref_filepath,
        shapes_gdf = shapes_gdf,
    )

    cropped_src_filepath = utils.add_epochs_prefix(
        filepath = src_filepath,
        prefix = 'cropped_',
        new_folderpath = working_dir,
    )

    crop_and_save(
        src_filepath = src_filepath,
        dst_filepath = cropped_src_filepath,
        shapes_gdf = shapes_gdf,
    )

    utils.resample_tif(
        ref_filepath = cropped_ref_filepath,
        src_filepath = cropped_src_filepath,
        dst_filepath = dst_filepath,
        resampling = resampling,
        src_nodata = src_nodata,
        dst_nodata = dst_nodata,
        dst_dtype = dst_dtype,
    )

    if delete_temp_files:
        os.remove(cropped_ref_filepath)
        os.remove(cropped_src_filepath)

In [ ]:
cropped_geoglam_maize_geotiff = utils.modify_filepath(
    filepath = geoglam_maize_geotiff,
    prefix = 'cropped_',
    new_folderpath = config.FOLDERPATH_OUTPUTS,
)

crop_and_resample(
    ref_filepath = config.FILEPATH_REFERENCE_GEOTIFF,
    src_filepath = geoglam_maize_geotiff,
    dst_filepath = cropped_geoglam_maize_geotiff,
    shapes_gdf = utils.get_actual_bounds_gdf(
        src_filepath = config.FILEPATH_REFERENCE_GEOTIFF,
        shapes_gdf = gpd.read_file(config.FILEPATH_ROI_SHAPE),
    ),
    resampling=rasterio.warp.Resampling.nearest,
    src_nodata=10001,
    dst_nodata=10001,
)

In [ ]:
with rasterio.open(cropped_geoglam_maize_geotiff) as src:
    print(src.meta)

In [ ]:
cropped_geoglam_cropmask_filepath = utils.modify_filepath(
    filepath = cropped_geoglam_maize_geotiff,
    prefix = 'cropmask_',
    new_folderpath = config.FOLDERPATH_MASKS,
)

cropped_geoglam_interp_filepath = utils.modify_filepath(
    filepath = cropped_geoglam_maize_geotiff,
    prefix = 'interp_',
    new_folderpath = config.FOLDERPATH_MASKS,
)

In [ ]:
create_binary_mask(
    tif_filepath = cropped_geoglam_maize_geotiff,
    out_mask_filepath = cropped_geoglam_cropmask_filepath,
    min_threshold = 10 * 100,
    how = 'gte',
)

create_binary_mask(
    tif_filepath = cropped_geoglam_maize_geotiff,
    out_mask_filepath = cropped_geoglam_interp_filepath,
    min_threshold = 0 * 100,
    how = 'gt',
)

In [ ]:
tif_filepaths

In [ ]:
geoglam_wc_cropmask_tif_filepath = os.path.join(config.FOLDERPATH_MASKS, 'geoglam_wc_maizemask.tif')

resample_worldcereal_cropmasks.merge_tifs(
    tif_filepaths = [
        binary_mask_filepaths['maize'],
        cropped_geoglam_cropmask_filepath,
    ],
    dst_filepath = geoglam_wc_cropmask_tif_filepath,
)

In [ ]:
geoglam_wc_interp_tif_filepath = os.path.join(config.FOLDERPATH_MASKS, 'geoglam_wc_interp.tif')

resample_worldcereal_cropmasks.merge_tifs(
    tif_filepaths = [
        binary_mask_filepaths['tc-annual_temporarycrops'],
        cropped_geoglam_interp_filepath,
    ],
    dst_filepath = geoglam_wc_interp_tif_filepath,
)